# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint

# Hackathon : Voice commands based food ordering system
The goal of the hacakthon is to train your model on different types of voice data (such as
clean studio data, noisy data and finally your own data)

### Setup Steps

In [0]:
import torch
from torch.autograd import Variable
import numpy as np
import librosa
import os
import warnings
from time import sleep
import sys
warnings.filterwarnings('ignore')

## Pretrained Network for deep features


The following function contains code to load a pre-trained network to produces deep features for the audio sample. This network is trained with delta MFCC features of mono channel 8000 bit rate audio sample.

In [0]:
def get_network():

    net = torch.nn.Sequential()

    saved_net = torch.load("net_speech_89.pt").cpu()

    for index, module in enumerate(saved_net):
        net.add_module("layer"+str(index),module)
        if (index+1)%17 == 0 :
            break
    return net

In [0]:
get_network()

Sequential(
  (layer0): Linear(in_features=900, out_features=800, bias=True)
  (layer1): ReLU()
  (layer2): Linear(in_features=800, out_features=700, bias=True)
  (layer3): ReLU()
  (layer4): Linear(in_features=700, out_features=600, bias=True)
  (layer5): ReLU()
  (layer6): Linear(in_features=600, out_features=500, bias=True)
  (layer7): ReLU()
  (layer8): Linear(in_features=500, out_features=400, bias=True)
  (layer9): ReLU()
  (layer10): Linear(in_features=400, out_features=300, bias=True)
  (layer11): ReLU()
  (layer12): Linear(in_features=300, out_features=200, bias=True)
  (layer13): ReLU()
  (layer14): Linear(in_features=200, out_features=100, bias=True)
  (layer15): ReLU()
  (layer16): Linear(in_features=100, out_features=50, bias=True)
)

##Obtaining Features from Audio samples
Generate features from a audio sample of '.wav' format
* Generate Delta MFCC features of order 1 and 2 
* Passes them through the above mentioned deep neural net
* the obtained deep features are returned

Parameters: Filepath (path of audio sample),
                       sr (sampling rate, all the samples provided are of 8000 bitrate)
         
  Caution: Do not change the default parameters

In [0]:
def get_features(filepath, sr=8000, n_mfcc=30, n_mels=128, frames = 15):
    
    
    y, sr = librosa.load(filepath, sr=sr)
    D = np.abs(librosa.stft(y))**2
    S = librosa.feature.melspectrogram(S=D)
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
    log_S = librosa.power_to_db(S,ref=np.max)
    features = librosa.feature.mfcc(S=log_S, n_mfcc=n_mfcc)
    if features.shape[1] < frames :
        features = np.hstack((features, np.zeros((n_mfcc, frames - features.shape[1]))))
    elif features.shape[1] > frames:
        features = features[:, :frames]
    # Find 1st order delta_mfcc
    delta1_mfcc = librosa.feature.delta(features, order=1)

    # Find 2nd order delta_mfcc
    delta2_mfcc = librosa.feature.delta(features, order=2)
    features = np.hstack((delta1_mfcc.flatten(), delta2_mfcc.flatten()))
    features = features.flatten()[np.newaxis, :]
    features = Variable(torch.from_numpy(features)).float()
    deep_net = get_network()
    deep_features = deep_net(features)
    #print(features.shape)
    #print(audio_file)
    #features.flatten()[np.newaxis, :]
    return deep_features.data.numpy().flatten()

## All the voice sample needed for training are present across the folders "Noisy_data" and "studio_data"

In [0]:
%ls

BLR_M2_Hackathon.ipynb        Noisy_data/              sample_data/
__MACOSX/                     noisy_data_features.sav  studio_data/
mlp_trained_model_noise.clf   noisy_data_labels.sav    studio_data_features.sav
mlp_trained_model_studio.clf  Noisy_data.zip           studio_data_labels.sav
net_speech_89.pt              Record_audio.py          studio_data.zip


##Stage 1: Loading data (5 Marks)

* Load 'Studio data' and extract features from the data

### Evaluation Criteria: 
* Complete the code in the load_data function
* The function should take path of the folder containing audio samples as input
* It should return features of all the audio samples present in the specified folder into single array (list of lists or 2-d numpy array) and their respective labels should be returned too

In [0]:
import glob
def load_files(folder_path):
  labels = []
  wav_files = []
  features = []
  # This loop is to store the labels and filenames into list
  for filename in glob.iglob(folder_path + '/*.wav'):
    # Store the labels of each wav file in a list
    labels.append(int((filename.split('/')[-1]).split('_')[0]))
    # Store the wav files in a list
    wav_files.append(filename)
  # This Loop is to get features and store into features list
  for filename in wav_files:
    features.append(get_features(filename))
  return np.asarray(features), np.asarray(labels)

In [0]:
!pwd

/content


####load data and labels from studio_data folder

In [0]:
studio_recorded_features, studio_recorded_labels = load_files('studio_data')
print(studio_recorded_features.shape, studio_recorded_labels.shape)

(8423, 50) (8423,)


In [0]:
import joblib
studio_recorded_features, studio_recorded_labels = np.asarray(joblib.load('studio_data_features.sav')),np.asarray(joblib.load('studio_data_labels.sav'))

In [0]:
from google.colab import files

joblib.dump(studio_recorded_features, 'studio_data_features.sav')               # Command to save the model file
joblib.dump(studio_recorded_labels, 'studio_data_labels.sav')

files.download("studio_data_features.sav")                                      # Download the model file to local PC
files.download("studio_data_labels.sav")                                        # Download the model file to local PC

In [0]:
studio_recorded_features.shape, studio_recorded_labels.shape

((8423, 50), (8423,))

## Stage 2: Training classifier on the studio_data (16 Marks)
* The goal here is to train your model on voice samples collected in a noiseless studio
setup above

### Evaluation Criteria: 
* Train the classifier, save the model
* The score you get: Validation accuracy percentage of 15 (Validation data should be at
least 20% of the total data)
* Example: If a team gets, 80 % accuracy on the validation set, then the marks will be
80% of 15 marks i.e. 12 marks (will round of the score, in case of non - integer scores) and deploy the (refer colab notebook)

#### Train a classifier on the features obtained from studio_data

In [0]:
# Import train_test_split header
from sklearn.model_selection import train_test_split

# Split files for training and testing (80:20)
X_train, X_test, y_train, y_test = train_test_split(studio_recorded_features, studio_recorded_labels, test_size=0.2, random_state=43)

# Train the Data as a numpy array
X_train = np.array(X_train) 
y_train = np.array(y_train)
X_train.shape, y_train.shape

((6738, 50), (6738,))

### Use the above data and train using MLPClassifier:

In [0]:
# Import Necessary Headers
from sklearn.neural_network import MLPClassifier

# Train the data
clf = MLPClassifier(random_state = 12)
clf.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=12, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [0]:

# Import Necessary Headers
from sklearn.neural_network import MLPClassifier

# Train the data
clf = joblib.load('mlp_trained_model_studio.clf')
clf.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=41, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

### Use this to predict accuracy for the trained model:

In [0]:
# Import necessary headers
from sklearn.metrics import accuracy_score

# Save the predicted values in a variable
predicted_values = clf.predict(np.array(X_test))

# Predict Accuracy score (in %)
accuracy_score(np.array(y_test),predicted_values)*100

88.24925816023739

####Save your model

Hint:
* Incase if you are using scikit learn model for training, you can use joblib package to save the model.
* Manually implemented models as a function or class can be saved using pickle 

In [0]:
import joblib                         # Import Necessary Headers

file = 'mlp_trained_model_studio.clf' # Specify filename of model file
joblib.dump(clf, file)                # Command to save the model file

['mlp_trained_model_studio.clf']

In [0]:
%ls -F $file                          # Check if the created model file exists!

MLP_trained_model_studio.clf


#### Download your trained model using the code below
* given the path of model file the following code downloads it through the browser

In [0]:
from google.colab import files
files.download(file)                  # Download the model file to local PC

## Stage 3: Deploy your classifier on the server (3 Marks)

* Deploy your model on the server, check the deployment instructions in the hackathon documentation for details

### Evaluation Criteria: 

* There are two stages in the food ordering application
        
    *   Ordering Item
    *   Providing the number of servings
    
* If both the stages are cleared with correct predictions you will get
complete marks
* Otherwise, no marks will be awarded



#### Now deploy the model trained on studio_data in the sever to order food correctly. 
#### Deployment instruction are given in the Hackathon documentation
#### After deploying and checking the application come back here to train on Noisy_data to generalise better in real situations

## Stage 4: Load 'Noisy_data', train a Classifier on the same and deploy (3 Marks)

* The goal here is to train your model on voice samples collected in a noisy environment and save the model

### Evaluation Criteria:

* Load 'Noisy_data'
* Train the classifier, save the model



#### load data and labels from Noisy_data folder

In [0]:
noisy_data, noisy_data_labels = load_files('Noisy_data')

In [0]:
noisy_data, noisy_data_labels = np.asarray(joblib.load('noisy_data_features.sav')),np.asarray(joblib.load('noisy_data_labels.sav'))

In [0]:
noisy_data.shape, noisy_data_labels.shape

((11839, 50), (11839, 1))

In [0]:
type(noisy_data_labels)

numpy.ndarray

#### Train a classifier on the features obtained from noisy_data

In [0]:
# Split noise files for training and testing (80:20)
X_noise_train, X_noise_test, y_noise_train, y_noise_test = train_test_split(noisy_data, noisy_data_labels, test_size=0.2)

# Transform the Data as a numpy array and check shape
X_noise_train = np.array(X_noise_train) 
y_noise_train = np.array(y_noise_train)
X_noise_train.shape, y_noise_train.shape

# Train the data
clf_noise = MLPClassifier(random_state = 60)
clf_noise.fit(X_noise_train, y_noise_train)

# Save the predicted values in a variable
predicted_noise_values = clf_noise.predict(np.array(X_noise_test))

# Predict Accuracy score (in %)
accuracy_score(np.array(y_noise_test),predicted_noise_values)*100

29.43412162162162

####Save your model

Hint:
* Incase if you are using scikit learn model for training, you can use joblib package to save the model.
* Manually implemented models as a function or class can be saved using pickle 

In [0]:
noise_model_file = 'mlp_trained_model_noise.clf'        # Specify filename of model file
joblib.dump(clf_noise, noise_model_file)                # Command to save the model file

['mlp_trained_model_noise.clf']

In [0]:
%ls -F $noise_model_file                                # Check if the created model file exists!

mlp_trained_model_noise.clf


#### Download your trained model using the code below
* given the path of model file the following code downloads it through the browser

In [0]:
from google.colab import files
files.download(noise_model_file)

#### Now deploy the model trained on noisy_data in the sever to order food correctly.

## Stage 5: Use 'Noisy_data' and 'Studio-data' together, train a Classifier on the same and deploy (3 Marks)

* The goal here is to train your model on voice samples collected in a noisy environment 'and' studio-data save the model

### Evaluation Criteria:

* Use the 'Noisy_data' and 'studio-data' loaded above and train the classifier, save the model






#### Train a classifier on the features obtained from both the Noisy_data and Studio_data

In [0]:
studio_recorded_labels.shape, noisy_data_labels.shape

((8423,), (11839, 1))

In [0]:
# Concatinate both Studio and Noisy data to all data
all_features = np.vstack((studio_recorded_features, noisy_data))
all_labels = np.vstack((studio_recorded_labels.reshape(8423,1), noisy_data_labels))

# Split noise files for training and testing (80:20)
X_all_train, X_all_test, y_all_train, y_all_test = train_test_split(all_features, all_labels, test_size=0.2)

# Transform the Data as a numpy array and check shape
X_all_train = np.array(X_noise_train) 
y_all_train = np.array(y_noise_train)
X_all_train.shape, y_all_train.shape

# Train the data
clf_all = MLPClassifier()
clf_all.fit(X_all_train, y_all_train)

# Save the predicted values in a variable
predicted_all_values = clf_all.predict(np.array(X_all_test))
accuracy_score(np.array(y_all_test),predicted_all_values)*100

55.662472242783124

####Save your model

Hint:
* Incase if you are using scikit learn model for training, you can use joblib package to save the model.
* Manually implemented models as a function or class can be saved using pickle 

In [0]:
all_model_file = 'mlp_trained_model_all.clf'        # Specify filename of model file
joblib.dump(clf_all, all_model_file)                # Command to save the model file

['mlp_trained_model_all.clf']

#### Download your trained model using the code below
* given the path of model file the following code downloads it through the browser

In [0]:
from google.colab import files
files.download(all_model_file)

#### Now deploy the model trained on above in the sever to order food correctly. 
#### Deployment instruction are given the Hackathon documentation
#### After deploying and checking the application, record your teams data from the web application provided in the Hackathon document

In [0]:
!mkdir teamdata

#### Replace <YOUR_GROUP_ID> with your group id given in the lab

In [0]:
!wget -r -A .wav https://aiml-sandbox1.talentsprint.com/audio_recorder/<YOUR_GROUP_ID>/team_data/ -nH --cut-dirs=100  -P ./team_data

In [0]:
%ls

In [0]:
!unzip <zip_file_name>

## Stage 6: Collect the voice samples and refine the classifier trained on noisy_data, by now using your team’s data (10 Marks)


* The goal here is to refine the model that you trained on voice samples collected
in a noisy environment
* You will refine your model trained on noisy_data, save and download it.
* Deploy your model on the server, check deployment section in the same
document for details

### Evaluation Criteria:
* There are two stages in the food ordering application

    *   Ordering Item
    *   Providing the number of servings

* If both the stages are cleared with correct predictions you will get complete marks
* Otherwise, no marks will be awarded

#### Enhance the model trained with both the noisy data and studio_data to your team's voice samples

In [0]:
##YOUR CODE HERE

####  Now deploy the model trained above in the sever to order food correctly. 
#### Deployment instruction are given the Hackathon documentation